In [ ]:
import pybadlands_companion.toolSea as tools

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Building sea-level curve

The file (**CSV format**) provides for each line the following information:
* Time in years,
* Sea-level elevation for the considered time in meters.

In addition, the defined fluctuation times should be set in **increasing order starting from the oldest time**.

We create a sea level curve from scratch. For more details please refer to [/workspace/companion/seaLevel.ipynb](/workspace/companion/seaLevel.ipynb).

In [ ]:
sea = tools.toolSea()
sea.buildCurve(timeExt = [0.,30000000.], timeStep = 100000., seaExt = [-25.,-25.], 
                   ampExt = [-75.,-75.], periodExt = [10000000.,10000000.])
# Visualize
sea.plotCurves(fsize=(4,5), figName = 'Sea level')
# Export the sea-level file  to the data folder
sea.exportCurve(nameCSV='data/sealevel')

# Building tectonic files

Here, the tectonics (vertical displacements) include: 
* thermal subsidence on the continental margin; 
* waves of dynamic topogrpahy.

In [ ]:
# Parameter of model geometry
# Number of nodes along X-Y axis
nx = 701  
ny = 701  
xr = np.linspace(0., 700000., nx)
yr = np.linspace(0., 700000., ny)
# Space resolution along X-Y axis
dx = xr[1] - xr[0]  
dy = yr[1] - yr[0]  
# 
endTime = 20000000 #  Dynamic topogrpahy passes the domain in 20 My
dispTimeStep = 20000
stepNb = int(endTime/dispTimeStep)

## Thermal subsidence

In [ ]:
E0 = 3200.0  # m
tao = 62.8e6  # yr
dist = np.arange(0,150.0+1.0,1.0)  # continental margin
beta = 1.0/100.0*dist+1.0
r = (beta/np.pi)*np.sin(np.pi/beta)
subsidence_beta = E0*r*np.exp(0/tao) - E0*r*np.exp(-endTime/tao)
subsidence_beta = subsidence_beta*2/3
disp_sub = -subsidence_beta/stepNb  # rate of subsidence

## Dynamic topography

In [ ]:
# Parameter of dynamic topography (DT)
v = 0.05  # velocity of DT (m/yr)
l = 300000.  # wavelength (m)
zo = 100.  # amplitude (m)
# 
xr_DT = np.linspace(-300000., 300000., 301)
l1 = 200000
DT1 = zo*np.exp(-np.pi*(xr_DT)*(xr_DT)/l1/l1)

## Export the tect files

In [ ]:
z = np.zeros((ny,nx))
for k in range(0,stepNb):
    f = open("disp"+str(k)+".csv", "w")
    t = (k+1)*dispTimeStep

    disp = np.zeros((ny,nx))
    zn = np.zeros((ny,nx))
    disp = np.zeros((ny,nx))
    disp1 = np.ones((nx,ny)) * disp_sub[150]
    beta = np.zeros((nx,ny))
    r = np.zeros((nx,ny))

    posit = int(t*v/dx)
    if (posit<=int(l/dx)):
        for j in range(ny):
            for i in range(0,posit,1):
                zn[j,i] = DT1[(300-posit+1)+i]
                disp[j,i] = zn[j,i] - z[j,i]
    elif (posit<=int(nx)):
        posit_pass = posit - int(l/dx)
        for j in range(ny):
            for i in range(posit_pass,posit,1):
                zn[j,i] = DT1[i-posit_pass]
                disp[j,i] = zn[j,i] - z[j,i]
    else:
        posit_pass = posit - int(l/dx)
        for j in range(ny):
            for i in range(posit_pass,nx,1):
                zn[j,i] = DT1[i-posit_pass]
                disp[j,i] = zn[j,i] - z[j,i]
    z = np.copy(zn)

    for j in range(0,ny):
        for i in range(0,nx):
            x = (i-350)*dx
            y = (j-350)*dx
            if (x*x + y*y <= r2*r2):
                disp1[j][i] = 0.
            elif (x*x + y*y <= r3*r3):
                beta[j][i] = np.sqrt(x*x + y*y)/100000.0 - 1.0
                r[j][i] = (beta[j][i]/np.pi)*np.sin(np.pi/beta[j][i])
                disp1[j][i] = (-E0*r[j][i]*np.exp(0/tao) + E0*r[j][i]*np.exp(-endTime/tao))*2/3/stepNb
            f.write( str(disp[j][i]+disp1[j][i]) + "\n"  )
    f.close()

In [ ]:
# Our model runs to 30 Myr. 
# Since dynamic topogrpahy passes the domain in the first 20 Myr. 
# Therefore, from 20-30 Myr, only thermal subsidence is forced on the continental margin.

# Export the thermal subsidence for the last 10 Myr.
subsid_last = (-E0*r*np.exp(-20e6/tao)+E0*r*np.exp(-30e6/tao))*2/3
f = open(str(stepNb)+".csv", "w")
disp1 = np.ones((nx,ny)) * subsid_last[150]
beta = np.zeros((nx,ny))
r = np.zeros((nx,ny))
# 
for j in range(0,ny):
    for i in range(0,nx):
        x = (i-350)*dx
        y = (j-350)*dx
        if (x*x + y*y <= r2*r2):
            disp1[j][i] = 0.
        elif (x*x + y*y <= r3*r3):
            beta[j][i] = np.sqrt(x*x + y*y)/100000.0 - 1.0
            r[j][i] = (beta[j][i]/np.pi)*np.sin(np.pi/beta[j][i])
            disp1[j][i] = (-E0*r[j][i]*np.exp(-20e6/tao) + E0*r[j][i]*np.exp(-30e6/tao))*2/3
        f.write( str(disp1[j][i]) + "\n"  )
f.close()

In [ ]:
# Write tectonic section of the XML input file
for k in range(0,stepNb):
    print("<disp>")
    print("   <dstart>"+str(k*dispTimeStep)+"</dstart>")
    print("   <dend>"+str((k+1)*dispTimeStep)+"</dend>")
    print("   <dfile>data/disp"+str(k)+".csv</dfile>")
    print("</disp>")